In [1]:
from datasets import split_data, get_imagenet_dataset, get_class
from conformal_prediction import adaptive_conformal_score, get_calibration_scores, predict_conformal_set
from torch.utils.data import DataLoader
from torchvision.models import EfficientNet_B1_Weights, efficientnet_b1
import torch
import numpy as np

In [2]:
IMAGENET_PATH = 'datasets'
BATCH_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
model = efficientnet_b1(weights=EfficientNet_B1_Weights.IMAGENET1K_V1).to(device)
transform = EfficientNet_B1_Weights.IMAGENET1K_V1.transforms()

dataset = get_imagenet_dataset(IMAGENET_PATH, transform=transform)
test_dataset, calibrate_dataset = split_data(dataset, 0.02)

In [4]:
calibrate_loader = DataLoader(calibrate_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_scores = get_calibration_scores(model, calibrate_loader, adaptive_conformal_score, device)
print(calibration_scores.shape)

100%|██████████| 8/8 [00:19<00:00,  2.44s/it]

torch.Size([1000, 1])


In [5]:
alpha = 0.1
calib_len = len(calibration_scores)
threshold = calibration_scores.quantile(np.ceil((calib_len + 1) * (1 - alpha)) / calib_len, interpolation='higher')
print(threshold)

tensor(0.9981, device='cuda:0')


In [6]:
class_subset = get_class(test_dataset, 309)
class_loader = DataLoader(class_subset, batch_size=BATCH_SIZE, shuffle=False)
print(len(class_subset))

49


In [7]:
conformal_set = predict_conformal_set(model, class_loader, adaptive_conformal_score, threshold, device)

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


In [8]:
for i in conformal_set:
    print(len(i))

851
800
953
868
759
937
947
793
876
936
940
944
965
814
936
889
966
939
916
922
950
936
803
881
777
955
908
921
947
926
807
969
800
851
972
931
937
795
926
962
927
944
905
938
960
935
968
787
954
